### Question 3: How did the popular languages change from 2017 to 2022? 
We will look at the survey result from 2017 to 2022 to see how the popular languages change.


In [1]:
import pandas as pd
from collections import defaultdict
import plotly.express as px

In [2]:
# first, we need to create a list of unique languages
df = pd.read_csv('./survey_results_public_2022.csv')
worked_languages = df[['LanguageHaveWorkedWith']].value_counts().reset_index()
worked_languages.rename(columns={'LanguageHaveWorkedWith':'Language',0:'count'},inplace=True)
worked_languages_df = worked_languages['Language'].str.split(';', expand=True)
no_worked_languages_list = [*range(0,worked_languages_df.shape[1])]
language_list=[]
for i in no_worked_languages_list:
    language_list += worked_languages_df[i].unique().tolist()

In [5]:
# Define a function to prepare data for each year. 
def prep_df(link,col):
    '''
    INPUT
    link - the directory of the data file for each year
    col - the name of the column we want to slice the data on

    OUTPUT
    df_year - a new dataframe for each year with 2 columns: the languages and their value counts
    '''
    df = pd.read_csv(link)
    df_year = df[[col]].value_counts().reset_index()
    df_year.rename(columns={col:'Language',0:'count'},inplace=True)
    return df_year


In [4]:
# Call the prep_df function on each year's dataset. Note that the column name are different among the years.
year_2022 = prep_df('./survey_results_public_2022.csv','LanguageHaveWorkedWith')
year_2021 = prep_df('./survey_results_public_2021.csv','LanguageHaveWorkedWith')
year_2020= prep_df('./survey_results_public_2020.csv','LanguageWorkedWith')
year_2019= prep_df('./survey_results_public_2019.csv','LanguageWorkedWith')
year_2018= prep_df('./survey_results_public_2018.csv','LanguageWorkedWith')

/var/folders/3g/t2spg1m14bvcp_qpq4bgx3040000gn/T/ipykernel_4918/3235037720.py:3: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(link)


In [6]:
# Define a function to count the occurences of each language in each year's dataframe
def count(df,col1,col2,look_for):
    ''' 
    INPUT:
    df - the sliced dataframe as the result of value_counts method 
    col1 - the column with the combinations of values
    col2 - the column with the count value for the combination
    look_for - the list of unique values 

    OUTPUT:
    new_df - a new dataframe with 2 columns, 1 column is the unique value, 1 column is the count of that unique value
    '''
    new_df = defaultdict(int)
    for val in look_for:
        for idx in range(df.shape[0]):
            if (val is not None) and (val in df[col1][idx]):
                new_df[val] += int(df[col2][idx])   
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = [col1, col2]
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df

In [8]:
# Now call the count function on each year's dataframe to count the occurences of each language.
result_2022 =count(year_2022,'Language','count',language_list)
result_2021 =count(year_2021,'Language','count',language_list)
result_2020 =count(year_2020,'Language','count',language_list)
result_2019 =count(year_2019,'Language','count',language_list)
result_2018 =count(year_2018,'Language','count',language_list)

In [9]:
# Add the year value into each result dataframe
result_2022["Year"]="2022"
result_2021["Year"]="2021"
result_2020["Year"]="2020"
result_2019["Year"]="2019"
result_2018["Year"]="2018"

In [10]:
# Concatenate all the years' dataframe into one big combined dataframe
combined_df= pd.concat([result_2018,result_2019,result_2020,result_2021,result_2022])
# Filter only the top 10 languages.
top_10=['SQL','Java','Python','JavaScript','TypeScript','HTML/CSS','PHP','R','PowerShell','Rust']
combined_df = combined_df.loc[combined_df['Language'].isin(top_10)]

In [11]:
# Now we create a line chart to show the changes in top 10 languages' popularity from 2018 to 2022. 
fig = px.line(combined_df, x='Year', y='count', color='Language',title="Popularity of Top 10 Popular Languages from 2018-2022")
fig.show()